In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

data = pd.read_csv("dataset.csv")

image_paths = data["image_path"]
labels = data["artist_label"]

In [ ]:
def load_and_preprocess_images(image_paths, target_size=(128, 128)):
    images = []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        images.append(img_array)
    
    return np.array(images)

# Load and preprocess images
X = load_and_preprocess_images(image_paths)
X = X / 255.0  # Normalize the images to [0, 1] range

# Convert labels to numpy array and one-hot encode them
y = np.array(labels)
y = to_categorical(y, num_classes=3)  # 3 classes (0, 1, 2)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Define a Convolutional Neural Network (CNN) model for multi-class classification
model = tf.keras.Sequential([
    # First convolutional block
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Second convolutional block
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Third convolutional block
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Flatten the output
    tf.keras.layers.Flatten(),
    
    # Fully connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    # Output layer with 3 units for multi-class classification
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes, softmax activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Categorical cross-entropy for multi-class classification
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

# Predict on the test set
y_pred = np.argmax(model.predict(X_test), axis=1)  # Convert softmax probabilities to class labels

# Get the actual class labels (as integers)
y_true = np.argmax(y_test, axis=1)

# Classification report for performance evaluation
report = classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1', 'Class 2'])
print(report)

c:\Users\Chien Shyong\Documents\School Work\Term 8\Data Science\yukien\myvenv\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
c:\Users\Chien Shyong\Documents\School Work\Term 8\Data Science\yukien\myvenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 11s 151ms/step - accuracy: 0.5966 - loss: 0.8448 - val_accuracy: 0.7988 - val_loss: 0.5292
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 148ms/step - accuracy: 0.7683 - loss: 0.5821 - val_accuracy: 0.8111 - val_loss: 0.5255
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 149ms/step - accuracy: 0.8167 - loss: 0.4923 - val_accuracy: 0.8686 - val_loss: 0.4022
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 142ms/step - accuracy: 0.8864 - loss: 0.3233 - val_accuracy: 0.8891 - val_loss: 0.3328
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 145ms/step - accuracy: 0.9194 - loss: 0.2421 - val_accuracy: 0.8665 - val_loss: 0.3451
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 8s 137ms/step - accuracy: 0.9267 - loss: 0.2160 - val_accuracy: 0.8871 - val_loss: 0.3214
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 140ms/step - accuracy: 0.9388 - loss: 0.1753 - val_accuracy: 0.9138 - val_loss: 0.2730
Epoch 8/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 139ms/step - accuracy: 0.9495 - loss: 0.1431 - val_accuracy: 0